In [3]:
!git clone https://github.com/Tanveer2719/NIDS_Coursework.git
%cd /kaggle/working/NIDS_Coursework/My_Code

Cloning into 'NIDS_Coursework'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 115 (delta 38), reused 111 (delta 34), pack-reused 0 (from 0)
Receiving objects: 100% (115/115), 139.96 KiB | 5.18 MiB/s, done.
Resolving deltas: 100% (38/38), done.
/kaggle/working/NIDS_Coursework/My_Code


In [4]:
import pandas as pd
import numpy as np

path = "/kaggle/input/unsw-nb-15/UNSW-NB15_combined.csv"

df = pd.read_csv(path)

print(df.shape)
df = df.drop(columns=['id', 'label'])
print(df.shape)

print(f"🧾 Unique class labels: {df['attack_cat'].unique()}")
df['label'] = df['attack_cat'].apply(lambda x: 0 if x == 'Normal' else 1)

print(f'Unique values for label {df["label"].unique()}')

df = df.drop(columns=["attack_cat"])
print(f'new shape{df.shape}')

print(df.dtypes)


# --------------------------- perform preprocessing--------------

from preprocess import Preprocess

# separate the features and labels so that the labesl are not encoded
labels = df['label']
features = df.drop(columns=['label'])

pp = Preprocess()
processed_features = pp.fit_transform_df_auto(df = features,n_categorical_levels=32, expected_categorical_format='onehot')

print(processed_features.shape)

# concat the features and the labels
processed_full_df = pd.concat([processed_features, labels], axis=1)


(257673, 45)
(257673, 43)
🧾 Unique class labels: ['Normal' 'Backdoor' 'Analysis' 'Fuzzers' 'Shellcode' 'Reconnaissance'
 'Exploits' 'DoS' 'Worms' 'Generic']
Unique values for label [0 1]
new shape(257673, 43)
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean   

In [5]:
# Filter out categorical one-hot encoded columns
numerical_columns = [col for col in processed_full_df 
                     if not (col.startswith("proto_") or 
                             col.startswith("service_") or 
                             col.startswith("state_") or
                             col == 'label')]

numerical_columns

['dur',
 'spkts',
 'dpkts',
 'sbytes',
 'dbytes',
 'rate',
 'sttl',
 'dttl',
 'sload',
 'dload',
 'sloss',
 'dloss',
 'sinpkt',
 'dinpkt',
 'sjit',
 'djit',
 'swin',
 'stcpb',
 'dtcpb',
 'dwin',
 'tcprtt',
 'synack',
 'ackdat',
 'smean',
 'dmean',
 'trans_depth',
 'response_body_len',
 'ct_srv_src',
 'ct_state_ttl',
 'ct_dst_ltm',
 'ct_src_dport_ltm',
 'ct_dst_sport_ltm',
 'ct_dst_src_ltm',
 'is_ftp_login',
 'ct_ftp_cmd',
 'ct_flw_http_mthd',
 'ct_src_ltm',
 'ct_srv_dst',
 'is_sm_ips_ports']

In [6]:
target = processed_full_df['label']
features = processed_full_df.drop(columns=['label'])

In [7]:
from record_level_embedding import RecordLevelEmbedder

embedder = RecordLevelEmbedder(selected_df = features, numerical_columns = numerical_columns, embed_dimension= 64)

embedded_df = embedder.transform_to_df()
embedded_df.head()

2025-07-18 12:57:33.281903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752843453.486409      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752843453.543517      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[INFO] Initializing RecordLevelEmbedder...
[INFO] Detecting categorical blocks...
[INFO] Detected 3 categorical blocks:
   - proto: 32 columns
   - service: 13 columns
   - state: 11 columns
[INFO] Embedder initialized with embedding dimension 64.
[INFO] Building Keras embedding model...
   - Adding numerical input: dur
   - Adding numerical input: spkts
   - Adding numerical input: dpkts
   - Adding numerical input: sbytes
   - Adding numerical input: dbytes
   - Adding numerical input: rate
   - Adding numerical input: sttl
   - Adding numerical input: dttl
   - Adding numerical input: sload
   - Adding numerical input: dload
   - Adding numerical input: sloss
   - Adding numerical input: dloss
   - Adding numerical input: sinpkt
   - Adding numerical input: dinpkt
   - Adding numerical input: sjit
   - Adding numerical input: djit
   - Adding numerical input: swin
   - Adding numerical input: stcpb
   - Adding numerical input: dtcpb
   - Adding numerical input: dwin
   - Adding nume

I0000 00:00:1752843465.407030      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752843465.407688      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


[INFO] Projection complete. Output embedding shape: (64,)
[INFO] Model build complete.
[INFO] Preparing inputs for embedding. Number of rows: 257673
   - Prepared numerical input: dur
   - Prepared numerical input: spkts
   - Prepared numerical input: dpkts
   - Prepared numerical input: sbytes
   - Prepared numerical input: dbytes
   - Prepared numerical input: rate
   - Prepared numerical input: sttl
   - Prepared numerical input: dttl
   - Prepared numerical input: sload
   - Prepared numerical input: dload
   - Prepared numerical input: sloss
   - Prepared numerical input: dloss
   - Prepared numerical input: sinpkt
   - Prepared numerical input: dinpkt
   - Prepared numerical input: sjit
   - Prepared numerical input: djit
   - Prepared numerical input: swin
   - Prepared numerical input: stcpb
   - Prepared numerical input: dtcpb
   - Prepared numerical input: dwin
   - Prepared numerical input: tcprtt
   - Prepared numerical input: synack
   - Prepared numerical input: ackdat
  

I0000 00:00:1752843467.218034     113 service.cc:148] XLA service 0x7a046800ccc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752843467.218780     113 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752843467.218798     113 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752843467.255391     113 cuda_dnn.cc:529] Loaded cuDNN version 90300


  50/8053 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step    

I0000 00:00:1752843467.561740     113 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8053/8053 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step
[INFO] Embedding complete. Output shape: (257673, 64)
[INFO] Embedding DataFrame ready.


,embed_0,embed_1,embed_2,embed_3,embed_4,embed_5,embed_6,embed_7,embed_8,embed_9,...,embed_54,embed_55,embed_56,embed_57,embed_58,embed_59,embed_60,embed_61,embed_62,embed_63
0,0.323895,0.280347,0.155832,-0.053439,-0.361903,0.233186,-0.437445,0.311803,0.142125,-0.417356,...,-0.255222,0.260181,-0.286322,-0.517430,0.971340,0.447956,0.450614,-0.285490,0.465887,-0.867367
1,0.332912,0.299343,-0.048359,-0.152607,-0.479104,0.062692,-0.356306,0.179040,0.264700,-0.519279,...,-0.456512,0.295706,-0.095546,-0.635053,0.984102,0.625872,0.322457,-0.530800,0.745944,-0.732511
2,0.342753,0.431685,0.047885,-0.009303,-0.371551,0.130810,-0.269249,0.201196,0.190147,-0.467690,...,-0.423929,0.182839,-0.181515,-0.600874,1.041268,0.678868,0.313919,-0.517422,0.650370,-0.918058
3,0.312977,0.169727,-0.170602,-0.247739,-0.144873,0.497168,-0.026276,0.577544,0.277047,-0.509056,...,-0.489411,-0.197837,0.005687,-0.595310,0.827662,0.309875,0.222740,-0.514070,0.484001,-1.085373
4,0.465762,0.419033,-0.188420,0.063205,-0.419376,0.006288,-0.014827,0.316691,0.325092,-0.421965,...,-0.235845,0.444564,0.006748,-0.631722,0.941018,0.758031,0.482254,-0.534847,0.599060,-0.577112


In [8]:
# Binary classification
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model


print(processed_full_df['label'].unique())

binary_target = (processed_full_df['label'] != 0).astype(int).values
unique, counts = np.unique(binary_target, return_counts=True)
print("Classes and their counts:")
for cls, cnt in zip(unique, counts):
    print(f"Class {cls}: {cnt}")


# -------------------------Model Train----------------
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from transformer import BasicTransformer
from classification import LastTokenClassificationHead
from sklearn.model_selection import train_test_split



# input data
X_embed = embedded_df.values                   # shape: (num_samples, embed_dim)
X_embed_seq = tf.expand_dims(X_embed, axis=1)  # shape: (batch_size, 1, embed_dim)

input_layer = Input(shape=(1, X_embed.shape[1]), name="record_input")

# Create transformer
transformer = BasicTransformer(
    n_layers=2,
    internal_size=128,
    n_heads=4,
    verbose=True  # See progress!
)


x = transformer.apply(input_layer, training=True)


# Create head
classification_head = LastTokenClassificationHead()

# Pool last token
x = classification_head.apply(x)

#  Add MLP head  
for i, layer_size in enumerate([128]):
    x = Dense(layer_size, activation="relu", name=f"classification_mlp_{i}_{layer_size}")(x)
    x = Dropout(0.1)(x)

#  output layer for binary classification
output_layer = Dense(1, activation="sigmoid", name="binary_output")(x)

# Build model  
model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

# Compile model 
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"],
    jit_compile=True
)

# split data

X_np = X_embed_seq.numpy() if hasattr(X_embed_seq, 'numpy') else np.array(X_embed_seq)


X_train, X_val, y_train, y_val = train_test_split(
    X_np, binary_target, test_size=0.2, random_state=42, stratify=binary_target
)

# Train
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)


[0 1]
Classes and their counts:
Class 0: 93000
Class 1: 164673
[BasicTransformer] Applying Encoder Block 1/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Applying Encoder Block 2/2
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[BasicTransformer] Final output shape: (None, 1, 64)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ record_input (InputLayer)       │ (None, 1, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_0_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block_1_transformer_encoder     │ (None, 1, 64)          │       149,504 │
│ (TransformerEncoderBlock)       │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ slice_last (Lambda)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ classification_mlp_0_128        │ (None, 128)            │         8,320 │
│ (Dense)                         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ binary_output (Dense)           │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 307,457 (1.17 MB)

 Trainable params: 307,457 (1.17 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8956 - loss: 0.2068[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 0.8956 - loss: 0.2068 - val_accuracy: 0.9165 - val_loss: 0.1641
Epoch 2/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9118 - loss: 0.1724 - val_accuracy: 0.9186 - val_loss: 0.1636
Epoch 3/10
6442/6442 ━━━━━━━━━━━━━━━━━━━━ 25s 4ms/step - accuracy: 0.9137 - loss: 0.1688 - val_accuracy: 0.9121 - val_loss: 0.1703
Epoch 4/10
6442/6442 

In [9]:
# evaluate the model
results = model.evaluate(X_val, y_val)
print(f"Validation loss: {results[0]:.4f}, Validation accuracy: {results[1]:.4f}")


from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

# Predict probabilities
y_pred_prob = model.predict(X_val)
y_pred = (y_pred_prob >= 0.5).astype(int).flatten()

# F1
f1 = f1_score(y_val, y_pred)
print(f"F1 Score: {f1:.4f}")

# Confusion matrix
cm = confusion_matrix(y_val, y_pred, labels=[0, 1])
print("Confusion Matrix:\n", cm)

if cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
    detection_rate = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    print(f"Detection Rate (Recall): {detection_rate:.4f}")
    print(f"False Alarm Rate: {false_alarm_rate:.4f}")
else:
    print("Confusion matrix does not contain both classes!")
    print("Check your labels: y_val:", np.unique(y_val), " y_pred:", np.unique(y_pred))


1611/1611 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9312 - loss: 0.1399
Validation loss: 0.1402, Validation accuracy: 0.9319
[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
[EncoderBlock] Input shape: (32, 1, 64)
[EncoderBlock] Output shape: (32, 1, 64)
1609/1611 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
[EncoderBlock] Input shape: (None, 1, 64)
[EncoderBlock] Output shape: (None, 1, 64)
1611/1611 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
F1 Score: 0.9464
Confusion Matrix:
 [[17082  1518]
 [ 1990 30945]]
Detection Rate (Recall): 0.9396
False Alarm Rate: 0.0816
